In [1]:
import math
import numpy as np
import timeit
import matplotlib.pyplot as plt
from ipywidgets import * # ez kell az interaktiv plottolashoz
from mpl_toolkits import mplot3d # a harom dimenzios plottolashoz kell
from IPython.display import display # a save buttonhoz

start = timeit.default_timer()

e_tolerance=0.1
a_tolerance=0.1
ink_tolerance=0.05
tau_tolerance=10.0
w_tolerance=0.01
om_tolerance=0.01

with open ("id.txt", "r") as myfile:
    id = myfile.read().replace("\n", ',')
id = id.split(',')

data=np.loadtxt('data.txt')

e = data[:,0]
a = data[:,1]
tau = data[:,5]
ink = data[:,2]
om = data[:,3]
w = data[:,4]

length = len(a)
solution_indices = []

for i in range (length):
    w[i] = math.radians(w[i])
    om[i] = math.radians(om[i])
    ink[i] = math.radians(ink[i])


for i in range (length):
    for j in range (length):
        if j > i:
            a_distance = a[i] - a[j]
            if abs(a_distance) < a_tolerance:
                e_distance = e[i] - e[j]
                if abs(e_distance) < e_tolerance:
                    ink_distance = ink[i] - ink[j]
                    if abs(ink_distance) < ink_tolerance:
                        w_distance = w[i] - w[j]
                        if abs(w_distance) < w_tolerance:       
                            om_distance = om[i] - om[j]
                            if abs(om_distance) < om_tolerance:     
                                tau_distance = tau[i] - tau[j]
                                if abs(tau_distance) < tau_tolerance:
                                    solution_indices.append([i, j])

solution_indices = np.array(solution_indices)
print(solution_indices)
stop = timeit.default_timer()
num_pairs = len(solution_indices)

print('Time: ', stop - start)

[[ 4536 16771]
 [ 9098 12776]
 [16677 18715]]
Time:  238.69439975399996


In [2]:
a_res = []
e_res = []
ink_res = []
tau_res = []
om_res = []
w_res = []

for i in range (num_pairs):
    for j in range(2):
        a_res.append(a[solution_indices[i,j]])
        e_res.append(e[solution_indices[i,j]])
        ink_res.append(ink[solution_indices[i,j]])
        tau_res.append(tau[solution_indices[i,j]])
        om_res.append(om[solution_indices[i,j]])
        w_res.append(w[solution_indices[i,j]])
        
a_res = np.array(a_res)
e_res = np.array(e_res)
ink_res = np.array(ink_res)
om_res = np.array(om_res)
tau_res = np.array(tau_res)
w_res = np.array(w_res)

print(a_res)
print(e_res)
print(ink_res)
print(tau_res)
print(om_res)
print(w_res)

a_ = [] # ezek a beolvasott palyaelemek, ezeket a tomboket vegig beken hagyom
e_ = []
ink_ = []
tau_ = []
om_ = []
w_ = []

# ezeket a listakat azert hozom letre, mert a palyaelemek_res listaknak az elemeit modositani fogom a sebessegvaltoztatasokkal
# ezek segitsegevel majd vizsgalhato, hogy mennyit valtoztak a palyaelemek a modositastol
for i in range (num_pairs):
    for j in range(2):
        a_.append(a[solution_indices[i,j]])
        e_.append(e[solution_indices[i,j]])
        ink_.append(ink[solution_indices[i,j]])
        tau_.append(tau[solution_indices[i,j]])
        om_.append(om[solution_indices[i,j]])
        w_.append(w[solution_indices[i,j]])

[0.94616256 0.949279   2.10845451 2.11936811 1.01613704 1.0161756 ]
[0.3630685  0.36550273 0.48196079 0.52048561 0.14551524 0.14700247]
[0.08251853 0.08294285 0.1350604  0.13462633 0.23356911 0.23301244]
[2458448.72431928 2458451.92238071 2458359.78944238 2458350.75000525
 2458519.52268527 2458516.85544996]
[3.40453107 3.40382335 1.5014735  1.50241002 0.04768867 0.04916772]
[5.32340559 5.31693871 3.71885358 3.7205657  2.39074211 2.38897019]


In [16]:
# Create a module from these functions
# szukseges fuggvenyek definialasa:

v = np.linspace(0,2*np.pi,1000)
v_len = len(v)
k = 0.01720209895
mu = k**2

def save(b):
    np.savetxt("modositott_palyaelemek.txt", b)

def parameter (a, e):
    p = a*(1-e**2)
    return p

def focal_equation (p, e, v):
    return p/(1+e*np.cos(v))

def eta (r, v):
    return r*np.sin(v)

def xsi (r, v):
    return r*np.cos(v)

def P_x (w, om, i):
    return np.cos(w)*np.cos(om)-np.sin(w)*np.sin(om)*np.cos(i)

def P_y (w, om, i):
    return np.cos(w)*np.sin(om)+np.sin(w)*np.cos(om)*np.cos(i)

def P_z (w, om, i):
    return np.sin(w)*np.sin(i)

def Q_x (w, om, i):
    return -np.sin(w)*np.cos(om)-np.cos(w)*np.sin(om)*np.cos(i)

def Q_y (w, om, i):
    return -np.sin(w)*np.sin(om)+np.cos(w)*np.cos(om)*np.cos(i)

def Q_z (w, om,i):
    return np.cos(w)*np.sin(i)

def fn_xyz (w, om, ink, r, v):
    matrix = [[P_x(w,om,ink), Q_x(w,om,ink)], [P_y(w,om,ink), Q_y(w,om,ink)], [P_z(w,om,ink), Q_z(w,om,ink)]]
    vector = [xsi(r,v), eta(r,v)]
    xyz = np.matmul(matrix, vector)
    return xyz

def fn_E(E,e,M):
    fn_E = E - e * np.sin(E) - M
    return fn_E

def fn_der_E(E,e):
    return 1-e*np.cos(E)

def E_0(M,e):
    return M + ((e*np.sin(M))/(1-np.sin(M+e)+np.sin(M)))

def difference_iteration(E0, e, M):
    return E0 - fn_E(E0, e, M) / fn_der_E(E0, e)

def true_anomaly_from_E(E, e):
    tol = 1e-7
    if abs(E - np.pi) < tol:
        return np.pi
    if abs(E) < tol:
        return 0
    if abs(E - 3*np.pi/2) < tol:
        return 3*np.pi/2
    else:
        return 2 * math.atan(math.sqrt((1+e)/(1-e))*math.tan(E/2))
    
def eccentric_anomaly_from_v(v,e):
    tol = 1e-7
    if abs(v) < tol:
        return 0
    else:
        return 2 * math.atan(math.sqrt((1-e)/(1+e))*math.tan(v/2))

def xsi_deriv (v, e, a):
    p = a*(1-e**2)
    return -math.sqrt(mu/p)*np.sin(v)

def eta_deriv (v, e, a):
    p = a*(1-e**2)
    return math.sqrt(mu/p)*(np.cos(v)+e)

def fn_velocity_xyz (a, e, w, om, ink, v):
    matrix = [[P_x(w,om,ink), Q_x(w,om,ink)], [P_y(w,om,ink), Q_y(w,om,ink)], [P_z(w,om,ink), Q_z(w,om,ink)]]
    vector = [xsi_deriv(v, e, a), eta_deriv(v, e, a)]
    v_xyz = np.matmul(matrix, vector)
    return v_xyz

def fn_c(xyz, v_xyz):
    return np.cross(xyz, v_xyz)

def fn_c_abs(c):
    return math.sqrt(c[0]**2 + c[1]**2 + c[2]**2)

def fn_v_abs(v_xyz):
    return (v_xyz[0])**2 + (v_xyz[1])**2 + (v_xyz[2])**2

def fn_h (v_xyz_square, r):
    return 0.5 * (v_xyz_square) - mu / r

def fn_lambda(r, v_xyz, xyz, c):
    return -mu / r * xyz + np.cross(v_xyz, c)

def fn_lambda_abs(lambda_vec):
    return math.sqrt(lambda_vec[0]**2 + lambda_vec[1]**2 + lambda_vec[2]**2)

def fn_inclination(c, c_abs):
    return math.asin(math.sqrt(c[0]**2+c[1]**2)/c_abs)
    
def fn_longitude_ascending_node(c):
    om = math.atan2(c[0] , c[1]) 
    return np.pi - om
    
def fn_semi_major_axis(h):
    return -mu/(2*h) 

def fn_eccentricity(h, c_abs):   
    return math.sqrt(1 + 2 * h * (c_abs / mu)**2)

def fn_argument_pericenter(c_abs, c, lambda_vec):
    w = math.atan2((c_abs*lambda_vec[2]), ((c[0]*lambda_vec[1])-(c[1]*lambda_vec[0])))
    if w < 0:
        return w + 2*np.pi
    else:
        return w 

def fn_time_pericenter(JD, e, E, a):
    return JD-(E - e*np.sin(E))/math.sqrt(mu/a**3)
    
def fn_true_anomaly(e, M):
    
    tol = 1e-7
    E_null_JD_original = 1
    E_JD_original = []
    v_JD_original = []
    
    for i in range (num_pairs):
        while True:
            E_next_JD_original = difference_iteration(E_null_JD_original, e, M)
            if abs(E_next_JD_original - E_null_JD_original) > tol:
                E_null_JD_original = E_next_JD_original
            else:
                E_JD_original.append(E_null_JD_original)
                break

    for i in range (num_pairs):
        if E_JD_original[i] < 0:
            E_JD_original[i] = E_JD_original[i] + 2 * np.pi
            
    for i in range (num_pairs):
        v_JD_original.append(true_anomaly_from_E((E_JD_original[i]), e))
        
    if v_JD_original[i] < 0:
        v_JD_original[i] = v_JD_original[i] + 2 * np.pi
    
    return v_JD_original

In [29]:
def plot_slider_with_velocity_JD(vx, vy, vz, JD_boost):
    vx = vx/1e6
    vy = vy/1e6
    vz = vz/1e6
   
    # az objektumparok egyike, ezeken nem valtoztatok:
    
    E_null_JD_original = 1
    M_JD_original = []
    E_JD_original = []
    v_JD_original = []
    tol = 1e-7
    
    for i in range (num_pairs): 
        M_JD_original.append(math.sqrt(mu) * a_res[2*i]**(-3/2) * (JD_boost - tau_res[2*i]))
    M_JD_original = np.array(M_JD_original) #kozepanomalia a sliderrel megadott JD-on
    
    for i in range (num_pairs):
        while True:
            E_next_JD_original = difference_iteration(E_null_JD_original, e_res[2*i], M_JD_original[i])
            if abs(E_next_JD_original - E_null_JD_original) > tol:
                E_null_JD_original = E_next_JD_original
            else:
                E_JD_original.append(E_null_JD_original)
                break

    for i in range (num_pairs):
        if E_JD_original[i] < 0:
            E_JD_original[i] = E_JD_original[i] + 2 * np.pi
            
    for i in range (num_pairs):
        v_JD_original.append(true_anomaly_from_E((E_JD_original[i]), e_res[2*i]))
        
    if v_JD_original[i] < 0:
        v_JD_original[i] = v_JD_original[i] + 2 * np.pi
        
    v_JD_original = np.array(v_JD_original) 
    
    xyz_JD_original = []
    r_JD_original = []
    r_original = []
    xyz_original = []

    for i in range (num_pairs):
        r_JD_original.append(focal_equation(parameter(a_res[2*i],e_res[2*i]), e_res[2*i], v_JD_original[i]))
        xyz_JD_original.append(fn_xyz(w_res[2*i],  om_res[2*i],  ink_res[2*i], r_JD_original[i], v_JD_original[i]))
        r_original.append(focal_equation(parameter(a_res[2*i], e_res[2*i]), e_res[2*i],v))
        xyz_original.append(fn_xyz(w_res[2*i],  om_res[2*i],  ink_res[2*i], r_original[i], v))
   
    xyz_original = np.array(xyz_original)
    xyz_JD_original = np.array(xyz_JD_original)

    
    # xyz,v_xyz az eredeti palyakon a paratlan szamu aszteroidara:
    # ameddig nem valtoztatom a sebessegeket:
    if vx == vy == vz == 0:
        
        M_JD = []
        E_JD = []
        v_JD = []
        E_null_JD = 1
        
        for i in range (num_pairs): 
            M_JD.append(math.sqrt(mu) * a_res[2*i+1]**(-3/2) * (JD_boost - tau_res[2*i+1]))
        M_JD = np.array(M_JD) #kozepanomalia a sliderrel megadott JD-on

        for i in range (num_pairs):
            while True:
                E_next_JD = difference_iteration(E_null_JD, e_res[2*i+1], M_JD[i])
                if abs(E_next_JD - E_null_JD) > tol:
                    E_null_JD = E_next_JD
                else:
                    E_JD.append(E_null_JD)
                    break

        for i in range (num_pairs):
            if E_JD[i] < 0:
                E_JD[i] = E_JD[i] + 2 * np.pi

        # a csuszkan beallitott JD-on a valodi anomalia, az excentrikus alapjan:
        for i in range (num_pairs):
            v_JD.append(true_anomaly_from_E((E_JD[i]), e_res[2*i+1]))
        if v_JD[i] < 0:
            v_JD[i] = v_JD[i] + 2 * np.pi

        v_JD = np.array(v_JD)
        
            
        xyz = []
        v_xyz = []
        r = []
        r_orbit = []
        xyz_orbit = []    
        for i in range (num_pairs):
            r.append(focal_equation(parameter(a_res[2*i+1], e_res[2*i+1]), e_res[2*i+1], v_JD[i]))
            xyz.append(fn_xyz(w_res[2*i+1],  om_res[2*i+1],  ink_res[2*i+1], r[i], v_JD[i]))
            v_xyz.append(fn_velocity_xyz(a_res[2*i+1], e_res[2*i+1], w_res[2*i+1],  om_res[2*i+1],  ink_res[2*i+1], v_JD[i]))
            r_orbit.append(focal_equation(parameter(a_res[2*i+1],e_res[2*i+1]),e_res[2*i+1],v))
            xyz_orbit.append(fn_xyz(w_res[2*i+1],  om_res[2*i+1],  ink_res[2*i+1], r_orbit[i], v))
                
        
        xyz = np.array(xyz)
        xyz_orbit = np.array(xyz_orbit)
        v_xyz = np.array(v_xyz)
        
        for n in range (num_pairs):
            fig = plt.figure()
            fig.set_size_inches(10, 10)
            ax = fig.add_subplot(111, projection='3d')
            ax.plot(xyz_original[n,0], xyz_original[n,1], xyz_original[n,2], color = 'green', linewidth='3', label = '1. aszteroida palyja')
            ax.plot(xyz_orbit[n,0], xyz_orbit[n,1], xyz_orbit[n,2], color = 'orange', linewidth='3', label = '2. aszteroida palyja')

            ax.scatter(0,0,0, color = 'black', marker = 'o', label = 'Nap')
            ax.scatter(xyz_JD_original[n,0], xyz_JD_original[n,1], xyz_JD_original[n,2], s = 50, color = 'blue', marker = 'o', label = '1. aszeroida')
            ax.scatter(xyz[n,0], xyz[n,1], xyz[n,2], s = 50, color = 'red', marker = 'o', label = '2. aszteroida')

            ax.legend(loc = 'upper right')  
            
    else:  
        
    # az uj palyaelemek alapjan excentrikus anomalia a csuszkan beallitott JD-on
    # kezdeti ertekek a palyaelemek_res ertekek:
        
        M_JD_dv= []
        E_JD_dv = []
        v_JD_dv = []
        E_null_JD_dv = 1
        
        for i in range (num_pairs): 
            M_JD_dv.append(math.sqrt(mu) * a_res[2*i+1]**(-3/2) * (JD_boost - tau_res[2*i+1]))
        M_JD_dv = np.array(M_JD_dv) #kozepanomalia a sliderrel megadott JD-on

        for i in range (num_pairs):
            while True:
                E_next_JD_dv = difference_iteration(E_null_JD_dv, e_res[2*i+1], M_JD_dv[i])
                if abs(E_next_JD_dv - E_null_JD_dv) > tol:
                    E_null_JD_dv = E_next_JD_dv
                else:
                    E_JD_dv.append(E_null_JD_dv)
                    break

        for i in range (num_pairs):
            if E_JD_dv[i] < 0:
                E_JD_dv[i] = E_JD_dv[i] + 2 * np.pi
        
        # a csuszkan beallitott JD-on a valodi anomalia, az excentrikus alapjan:
        for i in range (num_pairs):
            v_JD_dv.append(true_anomaly_from_E((E_JD_dv[i]), e_res[2*i+1]))
        if v_JD_dv[i] < 0:
            v_JD_dv[i] = v_JD_dv[i] + 2 * np.pi

        v_JD_dv = np.array(v_JD_dv)

# elso modositastol ez fog lefutni, ha JD-ket allitgatom v_res_at_slider_JD[i] fog valtozni   
        
        r_dv = []
        xyz_dv = []
        v_xyz_dv = []
        r_orbit_dv = []
        xyz_orbit_dv = []
        
        for i in range (num_pairs):
            r_dv.append(focal_equation(parameter(a_res[2*i+1], e_res[2*i+1]), e_res[2*i+1], v_JD_dv[i]))
            xyz_dv.append(fn_xyz(w_res[2*i+1],  om_res[2*i+1], ink_res[2*i+1], r_dv[i], v_JD_dv[i]))
            v_xyz_dv.append(fn_velocity_xyz(a_res[2*i+1], e_res[2*i+1], w_res[2*i+1], om_res[2*i+1], ink_res[2*i+1], v_JD_dv[i]))
            # a teljes modositott palya:
            r_orbit_dv.append(focal_equation(parameter(a_res[2*i+1], e_res[2*i+1]), e_res[2*i+1], v))
            xyz_orbit_dv.append(fn_xyz(w_res[2*i+1],  om_res[2*i+1], ink_res[2*i+1], r_orbit_dv[i], v))
        
        v_xyz_dv = np.array(v_xyz_dv)
        xyz_dv = np.array(xyz_dv)
        xyz_orbit_dv = np.array(xyz_orbit_dv)    
        r_dv = np.array(r_dv)
        
        # a sebessegeket itt adom hozza a v_xyz_dv tombhoz, az integralok mar ezekkel fognak dolgozni
        for i in range (num_pairs):
            v_xyz_dv[i,0] = v_xyz_dv[i,0]+vx
            v_xyz_dv[i,1] = v_xyz_dv[i,1]+vy
            v_xyz_dv[i,2] = v_xyz_dv[i,2]+vz
        
        c = []
        c_abs = []
        lambda_abs = []
        lambda_vec = []
        h_energy = []
        
        #c, lambda, h:
        for i in range (num_pairs):
            c.append(fn_c(xyz_dv[i], v_xyz_dv[i]))
        c = np.array(c)
      
        for i in range (num_pairs):
            lambda_vec.append(fn_lambda(r_dv[i], v_xyz_dv[i], xyz_dv[i], c[i]))
        lambda_vec = np.array(lambda_vec) # num_pairs x 3as tomb, elso indexeben num_pairs szamu lambda_x stb.
        
        v_xyz_square = []
        for i in range (num_pairs):
            v_xyz_square.append(fn_v_abs(v_xyz_dv[i]))
        v_xyz_square = np.array(v_xyz_square)
        
        for i in range(num_pairs):
            lambda_abs.append(fn_lambda_abs(lambda_vec[i]))
            h_energy.append(fn_h(v_xyz_square[i], r_dv[i]))
            c_abs.append(fn_c_abs(c[i]))
            
        lambda_abs = np.array(lambda_abs)
        h_energy = np.array(h_energy)
        c_abs = np.array(c_abs)
        
        for i in range (num_pairs):
            a_res[2*i+1] = (fn_semi_major_axis(h_energy[i])) 
            ink_res[2*i+1] = (fn_inclination(c[i], c_abs[i]))
            om_res[2*i+1] = (fn_longitude_ascending_node(c[i]))
            e_res[2*i+1] = (fn_eccentricity(h_energy[i], c_abs[i]))
            w_res[2*i+1] = (fn_argument_pericenter(c_abs[i], c[i], lambda_vec[i]))
            
        for i in range (num_pairs):
            tau_res[2*i+1] = (fn_time_pericenter(JD_boost, e_res[2*i+1], E_JD_dv[i], a_res[2*i+1]))
        

        for i in range(num_pairs):
            if e_res[2*i+1] >= 1:
                print('Asteroid number', i+1 ,'is no longer on an elliptical orbit!')

        # sebesseg modositast kovetoen a palya:
        
        r_next_JD = []
        xyz_next_JD = []
        v_xyz_next_JD = [] 
        r_orbit_next = []
        xyz_orbit_next = []
        
        # az itt kiszamitott palyaknal nem szamolok ujra, a sebessegmodositas utani v_xyz alapjan kapott 
        #palyaelemekbol ujra valodi anomaliat, mert a dv sebessegvaltoztatas pillanatszeruen tortenik,
        #addig a valodi anomalia nem fog megvaltozni (ezert marad itt is a v_JD_dv).
        
        for i in range (num_pairs):
            r_next_JD.append(focal_equation(parameter(a_res[2*i+1], e_res[2*i+1]), e_res[2*i+1],v_JD_dv[i]))
            xyz_next_JD.append(fn_xyz(w_res[2*i+1],  om_res[2*i+1], ink_res[2*i+1], r_next_JD[i], v_JD_dv[i]))
            v_xyz_next_JD.append(fn_velocity_xyz(a_res[2*i+1], e_res[2*i+1], w_res[2*i+1], om_res[2*i+1], ink_res[2*i+1], v_JD_dv[i]))
            # a teljes modositott palya:
            r_orbit_next.append(focal_equation(parameter(a_res[2*i+1], e_res[2*i+1]), e_res[2*i+1],v))
            xyz_orbit_next.append(fn_xyz(w_res[2*i+1],  om_res[2*i+1], ink_res[2*i+1], r_orbit_next[i], v))
     
        v_xyz_next_JD = np.array(v_xyz_next_JD)
        xyz_orbit_next = np.array(xyz_orbit_next)
        xyz_next_JD= np.array(xyz_next_JD)
        
        for n in range (num_pairs):
            fig = plt.figure()
            fig.set_size_inches(10, 10)
            ax = fig.add_subplot(111, projection='3d')
            ax.plot(xyz_original[n,0], xyz_original[n,1], xyz_original[n,2], color = 'green', linewidth='3', label = '1. aszteroida palyja (valtozatlan)')
            ax.plot(xyz_orbit_next[n,0], xyz_orbit_next[n,1], xyz_orbit_next[n,2], color = 'orange', linewidth='3', label = '2. aszteroida palyja (valtoztatott)')

            ax.scatter(0,0,0, color = 'black', marker = 'o', label = 'Nap')
            ax.scatter(xyz_JD_original[n,0], xyz_JD_original[n,1], xyz_JD_original[n,2], s = 50, color = 'blue', marker = 'o', label = '1. aszeroida (valtozatlan)')
            ax.scatter(xyz_next_JD[n,0], xyz_next_JD[n,1], xyz_next_JD[n,2], s = 50, color = 'red', marker = 'o', label = '2. aszteroida (valtoztatott)')

            ax.legend(loc = 'upper right')
        
  # a paronkent vett objektumok palyaelemeinek különbsége -> milyen közel vannak
        d_a = []
        d_a_original = []
        d_e = []
        d_e_original = []
        d_ink = []
        d_ink_original = []
        d_w = []
        d_w_original = []
        d_om = []
        d_om_original = []
        d_tau = []
        d_tau_original = []
        for i in range (num_pairs):
            d_a.append(abs(a_res[2*i+1]-a_res[2*i]))
            d_a_original.append(abs(a_[2*i+1]-a_[2*i]))
            d_e.append(abs(e_res[2*i+1]-e_res[2*i]))
            d_e_original.append(abs(e_[2*i+1]-e_[2*i]))
            d_ink.append(abs(ink_res[2*i+1]-ink_res[2*i]))
            d_ink_original.append(abs(ink_[2*i+1]-ink_[2*i]))
            d_w.append(abs(w_res[2*i+1]-w_res[2*i]))
            d_w_original.append(abs(w_[2*i+1]-w_[2*i]))
            d_om.append(abs(om_res[2*i+1]-om_res[2*i]))
            d_om_original.append(abs(om_[2*i+1]-om_[2*i]))
            d_tau.append(abs(tau_res[2*i+1]-tau_res[2*i]))
            d_tau_original.append(abs(tau_[2*i+1]-tau_[2*i]))
            
        print('A felnagytengelyek kulonbsege a sebessegvaltoztatas elott: \n', d_a_original, '\n',
              'A felnagytengelyek kulonbsege a sebessegvaltoztatas utan: \n', d_a, '\n',
              'Az excentricitasok kulonbsege a sebessegvaltoztatas elott: \n', d_e_original, '\n',
              'Az excentricitasok kulonbsege a sebessegvaltoztatas utan: \n', d_e, '\n',
              'Az inklinaciok kulonbsege a sebessegvaltoztatas elott: \n', d_ink_original, '\n',
              'Az inklinaciok kulonbsege a sebessegvaltoztatas utan: \n', d_ink, '\n',
              'A pericentrum argumentumok kulonbsege a sebessegvaltoztatas elott: \n', d_w_original, '\n',
              'A pericentrum argumentumok kulonbsege a sebessegvaltoztatas utan: \n',  d_w, '\n',
              'A felszallo csomo hosszak kulonbsege a sebessegvaltoztatas elott: \n', d_om_original, '\n',
              'A felszallo csomo hosszak kulonbsege a sebessegvaltoztatas utan: \n', d_om, '\n',
              'A pericentrum atmenetek idopontjanak kulonbsege a sebessegvaltoztatas elott: \n', d_tau_original, '\n',
              'A pericentrum atmenetek idopontjanak kulonbsege a sebessegvaltoztatas utan: \n', d_tau, '\n')
    
    save((a_res, e_res, ink_res, tau_res, om_res, w_res))


In [30]:
def objects_position_after_dv(JD): # a már ismertetett módon ábrázolja ez a függvény az objektumpárok pályáját
  # a transzfer után hívom meg, hogy látható lehessen utána milyen a pálya
  # a pillanatnyi távolságot kiírja a függvény, így vizsgálható, hogy mikor milyen messze vannak az objektumok.
    orb_elements=np.loadtxt('modositott_palyaelemek.txt')

    e_from_file = orb_elements[1,:]
    a_from_file = orb_elements[0,:]
    tau_from_file = orb_elements[3,:]
    ink_from_file = orb_elements[2,:]
    om_from_file = orb_elements[4,:]
    w_from_file = orb_elements[5,:]
    
    M_JD = []
    v_JD = []
    
    for i in range (2*num_pairs): 
        M_JD.append(math.sqrt(mu) * a_from_file[i]**(-3/2) * (JD - tau_from_file[i]))
    M_JD = np.array(M_JD)
    
    for i in range(2*num_pairs):
        v_JD.append(fn_true_anomaly(e_from_file[i], M_JD[i]))
    v_JD = np.array(v_JD)
    
    xyz_JD = []
    r_JD = []
    r = []
    xyz = []

    for i in range (2*num_pairs):
        r_JD.append(focal_equation(parameter(a_from_file[i],e_from_file[i]), e_from_file[i], v_JD[i]))
        xyz_JD.append(fn_xyz(w_from_file[i],  om_from_file[i],  ink_from_file[i], r_JD[i], v_JD[i]))
        r.append(focal_equation(parameter(a_from_file[i], e_from_file[i]), e_from_file[i],v))
        xyz.append(fn_xyz(w_from_file[i],  om_from_file[i],  ink_from_file[i], r[i], v))
   
    xyz = np.array(xyz)
    xyz_JD = np.array(xyz_JD)
    
    dist_x = []
    dist_y = []
    dist_z = []
    distance = []
    distance_km = []
    
    for i in range (0, 2*num_pairs, 2):
        dist_x.append(abs(xyz_JD[i,0]-xyz_JD[i+1,0]))
        dist_y.append(abs(xyz_JD[i,1]-xyz_JD[i+1,1]))
        dist_z.append(abs(xyz_JD[i,2]-xyz_JD[i+1,2]))
        
    for i in range(num_pairs):
        distance.append(math.sqrt(dist_x[i]**2+dist_y[i]**2+dist_z[i]**2))
        
    for i in range (num_pairs):    
        distance_km.append(distance[i] * 1.5e8)
        
    print('Az objektumok egymástól mért aktuális távolsága kiométerben:','\n', distance_km, '\n',
         'Az objektumok egymástól mért aktuális távolsága csillagászati egységben:','\n', distance)
    
    for n in range (0, 2*num_pairs, 2):
        fig = plt.figure()
        fig.set_size_inches(10, 10)
        ax = fig.add_subplot(111, projection='3d')
        ax.plot(xyz[n,0], xyz[n,1], xyz[n,2], color = 'green', linewidth='3', label = '1. aszteroida palyaja (valtozatlan)')
        ax.plot(xyz[n+1,0], xyz[n+1,1], xyz[n+1,2], color = 'orange', linewidth='3', label = '2. aszteroida palyaja (valtoztatott)')

        ax.scatter(0,0,0, color = 'black', marker = 'o', label = 'Nap')
        ax.scatter(xyz_JD[n,0], xyz_JD[n,1], xyz_JD[n,2], color = 'blue', marker = 'o', s = 50, label = '1. aszeroida (valtozatlan)')
        ax.scatter(xyz_JD[n+1,0], xyz_JD[n+1,1], xyz_JD[n+1,2], color = 'red', marker = 'o', s = 50, label = '2. aszeroida (valtoztatott)')
        ax.set_xlabel('X tengely')
        ax.set_ylabel('Y tengely')
        ax.set_zlabel('Z tengely')
        ax.legend(loc = 'upper right')

In [34]:
a_res = []
e_res = []
ink_res = []
tau_res = []
om_res = []
w_res = []

# azert kell, hogy a kovetkezo futtataskor a palyaelemek_res tombokbe ne a mar modositgatott ertekek legyenek
# hanem ujra az erdetik
# igy hogy ez itt van a fenti appendeles a palyaelemek_res-be nem is szukseges
for i in range (num_pairs):
    for j in range(2):
        a_res.append(a[solution_indices[i,j]])
        e_res.append(e[solution_indices[i,j]])
        ink_res.append(ink[solution_indices[i,j]])
        tau_res.append(tau[solution_indices[i,j]])
        om_res.append(om[solution_indices[i,j]])
        w_res.append(w[solution_indices[i,j]])

interact(plot_slider_with_velocity_JD,
         vx = BoundedFloatText(value = 0, min = -1000, max = 1000,  step = 1, description = 'vx * 1e-6'),
         vy = BoundedFloatText(value = 0, min = -1000, max = 1000,  step = 1, description = 'vy * 1e-6'),
         vz = BoundedFloatText(value = 0, min = -1000, max = 1000,  step = 1, description = 'vz * 1e-6'),
         JD_boost = FloatSlider(value =  2458615.174178 , min = 2458500, max = 2459500,  step = 10));

interactive(children=(BoundedFloatText(value=0.0, description='vx * 1e-6', max=1000.0, min=-1000.0, step=1.0),…

In [24]:
interact(plot_slider_with_velocity_JD,
         vx = BoundedFloatText(value = 0, min = -1000, max = 1000,  step = 1, description = 'vx * 1e-6'),
         vy = BoundedFloatText(value = 0, min = -1000, max = 1000,  step = 1, description = 'vy * 1e-6'),
         vz = BoundedFloatText(value = 0, min = -1000, max = 1000,  step = 1, description = 'vz * 1e-6'),
         JD_boost = FloatSlider(value = 0, min = 0, max = 5000,  step = 10));

interactive(children=(BoundedFloatText(value=0.0, description='vx * 1e-6', max=1000.0, min=-1000.0, step=1.0),…